DataExploration and Photo Level and Business level Feature Extraction Phase:-

This is photo level feature extraction and Business feature extraction step Using Inception-v3  pre-trained Classification model.Also we can use model from scratch but it will take long time to train also accuracy is bit low compare to pre-trained models.By using weights of Inception-v3 pretrained model  which was trained on ImageNet to will extract bottleneck features of Yelp photos.

In [1]:
import numpy as np; 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
from PIL import ImageFilter

In [ ]:
# Paths of train & test photos
train_path = "../input/data/train_photos/"
test_path = "../input/data/test_photos/"

# Paths of CSV files
train_pid_bid = '../input/data/train_photo_to_biz_ids.csv'
train_bid_label = '../input/data/train.csv'
test_pid_bid = '../input/data/test_photo_to_biz.csv'

# Make dataframes 
train_photos = pd.read_csv(train_pid_bid)
train_label = pd.read_csv(train_bid_label)
train_id = pd.read_csv(train_pid_bid) 
test_photos = pd.read_csv(test_pid_bid)

# Labels dictionary
label_notation = {0: 'good_for_lunch', 1: 'good_for_dinner', 2: 'takes_reservations',  3: 'outdoor_seating',
                  4: 'restaurant_is_expensive', 5: 'has_alcohol', 6: 'has_table_service', 7: 'ambience_is_classy',
                  8: 'good_for_kids'}

In [ ]:
print("Train...")
# Show 25 images in train set
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.subplots_adjust(wspace=0, hspace=0)

for x in range(25):
        plt.subplot(5, 5, x+1)
        im = Image.open(train_path + str(train_photos.photo_id[x]) + '.jpg')
        im = im.resize((192, 192), Image.ANTIALIAS)
        plt.imshow(im)
        plt.axis('off')

In [ ]:
print("Test...")
# Show 25 images in test set
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.subplots_adjust(wspace=0, hspace=0)

for x in range(25):
        plt.subplot(5, 5, x+1)
        im = Image.open(test_path +  str(test_photos.photo_id[x]) + '.jpg')
        im = im.resize((192, 192), Image.ANTIALIAS)
        plt.imshow(im)
        plt.axis('off')

Showing images for each label in training dataset for Analysis using matplot lib subplots.

In [ ]:
# Show 9 images belonged to each label
for l in label_notation:
    ids = train_attr[train_attr['labels'].str.contains(str(l))==True].business_id.tolist()[:9]
    plt.rcParams['figure.figsize'] = (7.0, 7.0)
    plt.subplots_adjust(wspace=0, hspace=0)
    for x in range(9):
        plt.subplot(3, 3, x+1)
        im = Image.open(train_path + str(train_photos.photo_id[ids[x]]) + '.jpg')
        im = im.resize((192, 192), Image.ANTIALIAS)
        plt.imshow(im)
        plt.axis('off')
    fig = plt.figure()
    fig.suptitle(label_notation[l])

In [ ]:
from tqdm import tqdm # Enable progress bar
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions # Load pre-trained model
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.layers import Flatten, Input

Evolution Of Convolution neural networks And Importance of using InceptionV3.

1.VGG16 and VGG19

This are two major drawbacks with VGGNet:
    It is painfully slow to train.
    The network architecture weights themselves are quite large (in terms of disk/bandwidth)
We still use VGG in many deep learning image classification problems.However, smaller network architectures are often more desirable (such as SqueezeNet, GoogLeNet, etc.).

2.resnet50

network-in-network architectures.

Draw back of resnet 50 model:- 

1.Even though ResNet is much deeper than VGG16 and VGG19, the model size is actually substantially smaller due to the usage of global average pooling rather than fully-connected layers — this reduces the model size down to 102MB for ResNet50.

3.Inception V3 Advantages compare to resnet:-

The goal of the inception module is to act as a “multi-level feature extractor” by computing 1×1, 3×3, and 5×5 convolutions within the same module of the network 

The weights for Inception V3 are smaller than both VGG and ResNet, coming in at 96MB.

In [ ]:
from keras.applications.inception_v3 import InceptionV3
# Use InceptionV3 model to extract bottleneck features
#ResNet_model = ResNet50(weights='imagenet', include_top = False)
base_model = InceptionV3(weights='imagenet', include_top=False)
# Construct a feature extractor based on pre-trained model
input = Input(shape=(224, 224, 3), name='image_input')
feature_extractor = base_model(input)
flattener = Flatten()(feature_extractor)
bottleneck_feature_extractor = Model(inputs=input, outputs=flattener)

# Empty arrays for storing extracted features
X_train = []; X_test = []

# Extract bottleneck features of photos for traninig
for i in tqdm(range(len(train_photos))):
    img_path = train_path + str(train_photos.photo_id[i]) + '.jpg'
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    bottleneck_features_train_raw = bottleneck_feature_extractor.predict(x)
    bottleneck_features_train_reduced =  bottleneck_features_train_raw.squeeze()
    X_train.append(bottleneck_features_train_reduced)

# Extract bottleneck features of photos for testing    
for i in tqdm(range(len(test_photos))):
    img_path = test_path + str(test_photos.photo_id[i]) + '.jpg'
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    bottleneck_features_test_raw = bottleneck_feature_extractor.predict(x)
    bottleneck_features_test_reduced =  bottleneck_features_test_raw.squeeze()
    X_test.append(bottleneck_features_test_reduced)

After Feature Extraction phase we need to concatinate features to photo_id.By Using pandas merged features and photo_ids in file name train_merge.csv and test_merge.csv file so that we can integrate to the Business model.


In [ ]:
# With extracted features, make a new train set by concatenating with photo_id in an original train set 
bottleneck_features_train = pd.DataFrame({'features' : X_train})
train_merge = pd.concat([bottleneck_features_train, train_photos], axis=1)
train_merge.to_pickle('train_merge.csv') # Save in pickle for preserving array format and considering its size

# With extracted features, make a new test set by concatenating with photo_id in an original test set 
bottleneck_features_test = pd.DataFrame({'features' : X_test})
test_merge = pd.concat([bottleneck_features_test, test_photos], axis=1)
test_merge.to_pickle('test_merge.csv') # Save in pickle for preserving array format and considering its size

In [ ]:
# Test loading data
train_merge = pd.read_pickle('train_merge.csv')
test_merge = pd.read_pickle('test_merge.csv')

# Check if the data frame is loaded normally
test_merge.head()

Step 2. Business feature manipulation

Feature pooling(mean,midean):This Method is Basically intended to Extract average features of photos in training set and test set  that are assigned to same business_id

In [ ]:
train_bid_features = pd.DataFrame(train_merge.groupby('business_id')['features'].apply(np.mean))
train_bid_features.reset_index(level=0, inplace=True)

test_bid_features = pd.DataFrame(test_merge.groupby('business_id')['features'].apply(np.mean))
test_bid_features.reset_index(level=0, inplace=True)

test_bid_features.head()

In [ ]:
# Sort labels by business_id in an ascending order, and then merge the label with mean feature of each business id
train_label = train_label.sort_values('business_id')
train_label = train_label.reset_index(drop=True)
train_bid_features['labels'] = train_label['labels']

# Find NaN label values and remove the row
nan_label_indices = pd.isnull(train_bid_features).any(1).nonzero()[0]
train_bid_features = train_bid_features.drop(train_bid_features.index[list(nan_label_indices)])

# Check if the data frame is well constructed
train_bid_features.head()

Last step in feature extraction model is to saving the Extracted features using pickle model.

In [ ]:
# Save the manipulated train & test data sets in pickle for preserving array format and considering its size
train_bid_features.to_pickle('train_set_inception_v3.csv')
test_bid_features.to_pickle('test_set_inception_v3.csv')